In [9]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import color
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os.path
import random
import json

In [10]:
import itertools
import sys
import cv2

In [11]:
meta_dir = r'C:\Users\Pundeer\Desktop\Data Science\Kaggle\amazon inventory reconciliation\amazon inventory\data\metadata'
img_dir = r'C:\Users\Pundeer\Desktop\Data Science\Kaggle\amazon inventory reconciliation\amazon inventory\data\bin-images'

In [12]:
N = range(10500)
num_samples = 4000
dataset_index = random.sample(N,num_samples)
#dataset_index
img_vec = []
meta_vec = []

for i in range(num_samples):
    fname_img = str(dataset_index[i]).zfill(5) + '.jpg'
    fname_meta = str(dataset_index[i]).zfill(5) + '.json'
    jpg_path = os.path.join(img_dir,fname_img)
    json_path = os.path.join(meta_dir,fname_meta)
    if os.path.isfile(jpg_path) & os.path.isfile(json_path):
        img_vec.append(fname_img)
        meta_vec.append(fname_meta)
        
print(np.shape(img_vec))
print(np.shape(meta_vec))

(3979,)
(3979,)


In [13]:
num_samples = len(img_vec)
#labels=np.zeros((num_samples,1))
labels = []
img_vec_less_5 = []
meta_vec_less_5 = []
for i in range(num_samples):
    fname_meta = meta_vec[i]
    fname_img = img_vec[i]
    json_path = os.path.join(meta_dir,fname_meta)
    f = open(json_path)
    dataset = json.load(f)
    expected_quantity = dataset['EXPECTED_QUANTITY']
    if expected_quantity <= 5:
        #print(fname)
        #print(expected_quantity)
        f.close()
        #labels[i,0]=expected_quantity
        labels.append(expected_quantity)
        img_vec_less_5.append(fname_img)
        meta_vec_less_5.append(fname_meta)
        
labels = np.array(labels, ndmin=2)
labels = labels.T
print(np.shape(labels))
print(np.shape(img_vec_less_5))
print(np.shape(meta_vec_less_5))

(2656, 1)
(2656,)
(2656,)


In [14]:
num_samples = len(meta_vec_less_5)
all_arrays = []
for i in range(num_samples):
    fname = img_vec_less_5[i]
    img_array = cv2.imread(os.path.join(img_dir, fname))
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    img_reshape = cv2.resize(img_array, (224,224))
    all_arrays.append(img_reshape)

In [15]:
X_ = []
Y_ = labels
for features in all_arrays:
    X_.append(features)
X_ = np.array(X_)

In [16]:
print(np.shape(X_), np.shape(Y_))

(2656, 224, 224, 3) (2656, 1)


In [44]:
x_train, x_test, y_train, y_test = train_test_split(X_, Y_, test_size=0.2)
x_train.shape

(2124, 224, 224, 3)

In [48]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train.shape

(2124, 6)

In [97]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

model=Sequential()
model.add(Conv2D(filters=8, kernel_size=(3,3), padding="Same", activation="relu", input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.05))

model.add(Conv2D(filters=16, kernel_size=(3,3), padding="Same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.05))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding="Same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.05))

model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(6,activation="softmax"))

#defining optimizer
optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.999)

#compile the model
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])

In [98]:
from keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(featurewise_center = False,                           #set input mean to 0
                           samplewise_center = False,                            #set each sample mean to 0
                           featurewise_std_normalization = False,                #divide input datas to std
                           samplewise_std_normalization = False,                 #divide each datas to own std
                           zca_whitening = False,                                #dimension reduction
                           rotation_range = 0.5,                                 #rotate 5 degree
                           zoom_range = 0.5,                                     #zoom in-out 5%
                           width_shift_range = 0.5,                              #shift 5%
                           height_shift_range = 0.5,
                           horizontal_flip = False,
                           vertical_flip = False
                          )
datagen.fit(x_train)

In [99]:
epochs = 10
batch_size = 50

history = model.fit_generator(datagen.flow(x_train,y_train, batch_size = batch_size), epochs = epochs,
                              validation_data = (x_test, y_test), steps_per_epoch = x_train.shape[0]//batch_size)

Epoch 1/10
42/42 [==============================] - 173s 4s/step - loss: 339.6956 - accuracy: 0.2174 - val_loss: 1.7870 - val_accuracy: 0.1327
Epoch 2/10
42/42 [==============================] - 149s 4s/step - loss: 2.0190 - accuracy: 0.2179 - val_loss: 1.7902 - val_accuracy: 0.1925
Epoch 3/10
42/42 [==============================] - 147s 4s/step - loss: 1.7703 - accuracy: 0.2203 - val_loss: 1.7886 - val_accuracy: 0.1963
Epoch 4/10
42/42 [==============================] - 148s 4s/step - loss: 1.7442 - accuracy: 0.2299 - val_loss: 1.7846 - val_accuracy: 0.1963
Epoch 5/10
42/42 [==============================] - 146s 3s/step - loss: 1.7108 - accuracy: 0.2366 - val_loss: 1.7813 - val_accuracy: 0.2000
Epoch 6/10
42/42 [==============================] - 146s 3s/step - loss: 1.7233 - accuracy: 0.2150 - val_loss: 1.7748 - val_accuracy: 0.2075
Epoch 7/10
42/42 [==============================] - 146s 3s/step - loss: 1.6911 - accuracy: 0.2395 - val_loss: 1.7668 - val_accuracy: 0.2037
Epoch 8/10


In [30]:
from keras.layers import Input, Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.optimizers import Adam

_input = Input((224,224,3)) 

conv1  = Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu")(_input)
conv2  = Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

flat   = Flatten()(pool4)
dense1 = Dense(512, activation="relu")(flat)
#drop5  = Dropout(0.25)(dense1)
output = Dense(6, activation="softmax")(dense1)

vgg12_model  = Model(inputs=_input, outputs=output)

In [31]:
#compile the model
vgg12_model.compile(optimizer = Adam(), loss = "categorical_crossentropy", metrics=["accuracy"])

In [63]:
from keras.preprocessing.image import ImageDataGenerator
datagen16 = ImageDataGenerator(featurewise_center = False,                         #set input mean to 0
                             samplewise_center = False,                            #set each sample mean to 0
                             featurewise_std_normalization = False,                #divide input datas to std
                             samplewise_std_normalization = False,                 #divide each datas to own std
                             zca_whitening = False,                                #dimension reduction
                             rotation_range = 0.5,                                 #rotate 5 degree
                             zoom_range = 0.5,                                     #zoom in-out 5%
                             width_shift_range = 0.5,                              #shift 5%
                             height_shift_range = 0.5,
                             horizontal_flip = False,
                             vertical_flip = False
                            )
datagen16.fit(x_train)

In [33]:
epochs = 5
batch_size = 50

#model fitting
history1 = vgg12_model.fit_generator(datagen16.flow(x_train,y_train, batch_size = batch_size), epochs = epochs,
                              validation_data = (x_test, y_test), steps_per_epoch = x_train.shape[0]//batch_size)

Epoch 1/5
42/42 [==============================] - 436s 10s/step - loss: 1.9013 - accuracy: 0.2218 - val_loss: 1.6352 - val_accuracy: 0.2049
Epoch 2/5
42/42 [==============================] - 490s 12s/step - loss: 1.6390 - accuracy: 0.2445 - val_loss: 1.6309 - val_accuracy: 0.2368
Epoch 3/5
42/42 [==============================] - 566s 13s/step - loss: 1.6354 - accuracy: 0.2594 - val_loss: 1.6164 - val_accuracy: 0.2406
Epoch 4/5
42/42 [==============================] - 541s 13s/step - loss: 1.6289 - accuracy: 0.2473 - val_loss: 1.6062 - val_accuracy: 0.2444
Epoch 5/5
42/42 [==============================] - 581s 14s/step - loss: 1.6123 - accuracy: 0.2493 - val_loss: 1.6064 - val_accuracy: 0.2312


#### CNN-LSTM

In [104]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, TimeDistributed, LSTM, GlobalAveragePooling2D
from keras.optimizers import Adam

model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=(1, 224, 224, 3)))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2),strides=(2,2))))

model.add(TimeDistributed(Conv2D(64, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2),strides=(2,2))))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='sigmoid'))
model.compile('adam', loss='categorical_crossentropy')

In [113]:
model.fit(x_train.reshape(2124, 1, 224,224,3), y_train, batch_size = 50, epochs = 5, verbose = 2)

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [ ]:
accuracy = model.evaluate(x_test.reshape(532,1,224,224,3), y_test, verbose=1)

160/532 [========>.....................] - ETA: 32s

In [123]:
print(model.history.history)

{'loss': [5.110257491104571, 5.097093937536193, 5.096988317881151, 5.09698826242975, 5.096988334494122]}


In [124]:
epochs = 5
batch_size = 50

#model fitting
history2 = model.fit_generator(datagen16.flow(x_train.reshape(2124, 1, 224,224,3) ,y_train, batch_size = batch_size), epochs = epochs,
                              validation_data = (x_test.reshape(532, 1, 224,224,3), y_test))

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (2124, 1, 224, 224, 3))